# Implementação com base de dados adquirida em aula

- Carregar dataset
- Aplicar filtros temporais
- Realizar segmentações (tempo | frequência)
- Criar vetores de características: VAR, RMS, ... (tempo) e FMD, FMN, ... (frequência)
- Aplicar 1 método de seleção de característica (p.e Select Kbest)
- Classificação (SVM)

# Carregamento da base de dados

In [126]:
from scipy import signal
import numpy as np

dataGabi = np.load("datasets/Gabi.npy")
dataJesse = np.load("datasets/Jesse.npy")

# Entendimento dos dados
dataGabi.shape, dataJesse.shape
# ((28, 1000, 4), (28, 1000, 4))

# Sempre tem 4 dados(4 eletrodos), os dois primeiros tem dados e os dois dps nao tem nada
# print(dataGabi)
# 4 -> eletrodo, canais, mas so estamos utilizando 2
    # Usando so 2 porque estamos fazendo uma diferença de potencial
# 28 -> classes, movimentos, comandos, amostras
# 1000 -> quantidade de pontos, 5segundos * 200hz

((28, 1000, 4), (28, 1000, 4))

In [127]:
# Correção do numero de eletrodos e unificação

# os dois primeiros eletrodos sao funcionais
dataGabi = dataGabi[:,:,:2]
dataJesse = dataJesse[:,:,:2]

# Entendimento dos dados
dataGabi.shape, dataJesse.shape
# ((28, 1000, 2), (28, 1000, 2))

data = np.array([dataGabi, dataJesse])
# data.shape
# (2, 28, 1000, 2)

# Esse transpose é so pra fazer o 1000 ir pra quarta posição
    # Para os dados que foram capturados sempre precisam ficar na posição final
    # O padrao de todas as funções do numpy scipy e tal é sempre em cima do axis -1 (ultima dimensao)
    # Padronização apeans
# Colocando os dados na dimensao final
data = data.transpose(0,1,3,2)
data.shape
# (2, 28, 2, 1000)

(2, 28, 2, 1000)

In [128]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
# import numpy as np
# from scipy import signal
# from sklearn.preprocessing import LabelEncoder
# import mne

# Funções para aplicação dos filtros temporais
def butter_bandpass(data, lowcut, highcut, fs=512, order=4):
    nyq = fs * 0.5
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='bandpass')
    return signal.filtfilt(b, a, data)


def butter_notch(data, cutoff, var=1, fs=512, order=4):
    nyq = fs * 0.5
    low = (cutoff - var) / nyq
    high = (cutoff + var) / nyq
    b, a = signal.iirfilter(order, [low, high], btype='bandstop', ftype="butter")
    return signal.filtfilt(b, a, data)

FS = 200

def print_graphs(data):
    for i in range(data.shape[0]):
        plt.plot(data[i,:])
    plt.title('Domínio do tempo')
    plt.show()

    # for i in range(data.shape[0]):
    #     plt.psd(data[i,:], Fs=FS)
    # plt.title('Domínio da frequência')
    # plt.show()
    
    # for i in range(data.shape[0]):
    #     plt.specgram(data[i,:], Fs=FS)
    # plt.title('Espectrograma')
    # plt.show()

# rcParams['figure.figsize'] = [15., 5.]
# print_graphs(X)

# Aplicação dos filtros

In [129]:
# Aplicação dos filtros
rcParams['figure.figsize'] = [15., 5.]
filtered_datas = []
for i in range(2):
    # print("Grafico do sujeito {i} antes do filtro")
    # print_graphs(data[i])
    # Função para aplicar os filtros de banda e notch
    def apply_filters(data, lowcut, highcut, notch_cutoff, notch_var=1, fs=200, order=4):
        # Aplicar o filtro de banda
        print(data[2][0])
        data = data.reshape(data.shape[1], data.shape[0], data.shape[2])
        data_bandpass = np.apply_along_axis(lambda x: butter_bandpass(x, lowcut, highcut, fs, order), axis=-2, arr=data)

        # Aplicar o filtro notch
        data_filtered = np.apply_along_axis(lambda x: butter_notch(x, notch_cutoff, notch_var, fs, order), axis=-2, arr=data_bandpass)

        return data_filtered

    # Parâmetros dos filtros
    lowcut = 0.5  # Freq. de corte inferior para o filtro de banda
    highcut = 50.0  # Freq. de corte superior para o filtro de banda
    notch_cutoff = 60.0  # Freq. central para o filtro notch
    
    # Aplicar os filtros aos dados
    filtered_datas.append(apply_filters(data[i], lowcut, highcut, notch_cutoff))

    # Verificar as dimensões dos dados filtrados
    print("Grafico do sujeito {i} depois do filtro")
    # print_graphs(filtered_data)

filtered_datas = np.array(filtered_datas)
filtered_datas = filtered_datas.reshape(filtered_datas.shape[0], filtered_datas.shape[2], filtered_datas.shape[1], filtered_datas.shape[3])
filtered_datas.shape


[207.31201172 300.27096558 232.62364197 174.98057556 276.13363647
 266.14437866 178.11834717 248.21528625 301.13113403 206.9828949
 215.2069397  303.43865967 236.83477783 185.6991272  286.6539917
 274.24124146 179.99203491 255.30613708 301.50137329 202.95127869
 212.32720947 307.8928833  237.69120789 178.72047424 282.51763916
 269.15872192 173.69778442 248.76506042 298.46832275 191.30149841
 213.87179565 299.00314331 232.45535278 176.88043213 284.46240234
 271.35031128 172.41873169 248.94830322 301.24707031 194.91796875
 213.86056519 307.38424683 241.44981384 180.36228943 289.8291626
 272.43487549 170.15609741 247.5383606  295.76065063 196.43637085
 202.34541321 296.66195679 228.68553162 178.07720947 282.98141479
 266.7277832  172.30653381 241.65924072 294.44793701 186.15539551
 202.9176178  297.63806152 226.59492493 170.75822449 281.31341553
 265.80029297 166.80140686 244.74839783 289.63842773 188.05900574
 199.75740051 291.05584717 230.07678223 177.78549194 284.5559082
 266.71658325 

(2, 28, 2, 1000)

# Segmentação

## Porque segmentar os dados em janelamento?
- Segmenta em mais informações para levar mais contextualização para o classificador

In [130]:
filtered_datas.shape
# data = data.reshape(data.shape[0] * data.shape[1], data.shape[2], data.shape[3])
# print(data.shape)
data = filtered_datas.reshape(filtered_datas.shape[0],filtered_datas.shape[1], filtered_datas.shape[2], filtered_datas.shape[3])
print(data.shape)

(2, 28, 2, 1000)


In [131]:
# Aplicação da segmentação (dom, tempo e frequencia)
from scipy.signal import stft

step = 29 # Se mudar isso aq da problema(nao vai bater o numero de janelas)
segment = 64
print("Shape inicial: ", data.shape)

n_win = int((data.shape[-1] - segment) / step) + 1
ids = np.arange(n_win) * step


# Janelas do dado no dominio do tempo
chunks_time = np.array([data[:,:,:,k:(k + segment)] for k in ids]).transpose(1, 2, 0, 3, 4)

_, _, chunks_freq = signal.stft(data, fs=200, nperseg=segment, noverlap=32)
chunks_freq = np.swapaxes(chunks_freq, 2, 3)

print('Formato (shape) dos dados depois da divisão de janelas')
print(f'Dominio do tempo: {chunks_time.shape} - (classes+ensaios, canais, janelas, linhas)')
print(f'Dominio da frequência:  {chunks_freq.shape} - (classes+ensaios, canais, janelas, linhas)')

# Quer dizer que tem 33 janelas com 64 pontos
# 33 * 64 = 2112
    # Faz sentido, porque temos 1000 pontos, ai como dividimos em 33 janelas de 29 passos, ele tem mais que o dobro realmente

# O da frequencia saiu diferente
    # Nao importa a quantidade de pontos na ultima dimensao, o que tem que bater é quantidade de janelas
    


Shape inicial:  (2, 28, 2, 1000)
Formato (shape) dos dados depois da divisão de janelas
Dominio do tempo: (2, 28, 33, 2, 64) - (classes+ensaios, canais, janelas, linhas)
Dominio da frequência:  (2, 28, 33, 2, 33) - (classes+ensaios, canais, janelas, linhas)


*Tarefa 2*: Separar os participantes e armazenar os dados em disco para execução das próximas tarefas.

In [132]:
data.shape

part_data = []

for i in range(2):
    part_data.append(data[i*28:(i+1)*28])
    np.save(f"datasets/part_data{i}.npy", part_data[i])




    np.save(f"datasets/participant_{i}_time.npy", chunks_time[i])
    np.save(f"datasets/participant_{i}_freq.npy", chunks_freq[i])





### Shape dos dados salvos

In [133]:
time_freq_1 = np.load("datasets/participant_0_time.npy")
time_freq_2 = np.load("datasets/participant_1_time.npy")
time_freq_1.shape, time_freq_2.shape


((28, 33, 2, 64), (28, 33, 2, 64))

In [134]:
freq_freq_1 = np.load("datasets/participant_0_freq.npy")
freq_freq_2 = np.load("datasets/participant_1_freq.npy")

freq_freq_1.shape, freq_freq_2.shape

((28, 33, 2, 33), (28, 33, 2, 33))